In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('../data/2020_Notebook02_train_output.csv')
test = pd.read_csv('../data/2020_Notebook02_test_output.csv')
train.drop('MONEDA',axis=1,inplace=True)
test.drop('MONEDA',axis=1,inplace=True)

train

,latitud,longitud,Barrio,Ciudad,Area_total,Area_constr,Area_constr_cat,NroBanios,Dormitorios,Antiguedad,...,restaurant_poi_count,school_poi_count,bus_station_poi_count,college_poi_count,university_poi_count,marketplace_poi_count,police_poi_count,local_average_bedrooms,local_average_total_area,Precio_m2_total
0,-12.091370,-77.012973,San Isidro,Lima,600.0,600.0,18.0,4.0,4.0,35.0,...,3.0,3.0,2.0,1.0,0.0,0.0,1.0,4.430264,450.449272,6212.500000
1,-12.110077,-76.965640,Santiago de Surco,Lima,1387.0,793.0,19.0,4.0,4.0,20.0,...,4.0,4.0,0.0,0.0,0.0,0.0,0.0,4.430264,450.254999,5969.718818
2,-12.079926,-76.923410,NoEspecifica,Lima,490.0,490.0,16.0,5.0,5.0,21.0,...,1.0,2.0,0.0,0.0,0.0,0.0,0.0,4.430017,450.476426,6946.312245
3,-12.207427,-76.937033,Villa el Salvador,Lima,140.0,140.0,1.0,3.0,6.0,16.0,...,35.0,15.0,0.0,0.0,0.0,2.0,0.0,4.429770,450.562824,2200.000000
4,-12.144977,-76.989592,Santiago de Surco,Lima,338.0,215.0,4.0,3.0,4.0,23.0,...,24.0,13.0,0.0,2.0,0.0,2.0,1.0,4.430583,450.444911,6286.390533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3641,-12.092924,-76.923858,La Molina,Lima,371.0,355.0,12.0,4.0,4.0,15.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,4.430264,450.505801,4528.301887
3642,-12.085432,-76.957820,La Molina,Lima,260.0,200.0,4.0,4.0,4.0,18.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,4.430264,450.533202,6057.692308
3643,-12.073441,-76.914661,La Molina,Lima,700.0,450.0,15.0,4.0,4.0,19.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.430264,450.424587,6750.000000
3644,-12.841678,-76.526003,Asia,Cañete,300.0,200.0,4.0,4.0,4.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.430477,450.686589,4980.000000


In [2]:
train.Barrio = train.Barrio.replace('Chosica (Lurigancho)','Chosica')
test.Barrio = test.Barrio.replace('Chosica (Lurigancho)','Chosica')

In [3]:
to_encoded_columns = train.select_dtypes(include='object').filter(regex='^(?!geohash)')
to_encoded_columns

,Barrio,Ciudad
0,San Isidro,Lima
1,Santiago de Surco,Lima
2,NoEspecifica,Lima
3,Villa el Salvador,Lima
4,Santiago de Surco,Lima
...,...,...
3641,La Molina,Lima
3642,La Molina,Lima
3643,La Molina,Lima
3644,Asia,Cañete


In [7]:
drop = test[test.Barrio == 'La Punta'].index
test.drop(drop, inplace=True)

In [8]:
#Estas columnas se van a encodear con LabelEncoder.
#Para los Geohash utilizaremos otros metodo.

to_encoded_columns = train.select_dtypes(include='object').filter(regex='^(?!geohash)')

#Creamos los artifacts.
import pickle
from sklearn.preprocessing import LabelEncoder
label_encoder_dict = {}

for column in to_encoded_columns:
    print(column)
    label_encoder = LabelEncoder()
    label_encoder.fit(train[column])
    label_encoder_dict[column] = label_encoder    
    train[column] = label_encoder.transform(train[column])
    test[column] = label_encoder.transform(test[column])
    
with open('../data/label_encoder_dict.pkl','wb') as handle:
    pickle.dump(label_encoder_dict, handle, protocol = pickle.HIGHEST_PROTOCOL)

Ciudad


#### Encodeo de Geohashes:

Para poder encodear los Geohashes tenemos que, como son variables de alta cardinalidad, tenemos que buscar un método para poder asignar geohashes cercanos a coordenadas nuevas que aparezcan cuando se haga la inferencia. 

In [9]:
#Por ahora no usaré geohashes. Veremos primero cómo va el performance con un modelo simple.
train.to_csv('../data/2020_Notebook03_train_output.csv',index=False)
test.to_csv('../data/2020_Notebook03_test_output.csv', index=False)

In [10]:
train

,latitud,longitud,Barrio,Ciudad,Area_total,Area_constr,Area_constr_cat,NroBanios,Dormitorios,Antiguedad,...,restaurant_poi_count,school_poi_count,bus_station_poi_count,college_poi_count,university_poi_count,marketplace_poi_count,police_poi_count,local_average_bedrooms,local_average_total_area,Precio_m2_total
0,-12.091370,-77.012973,39,2,600.0,600.0,18.0,4.0,4.0,35.0,...,3.0,3.0,2.0,1.0,0.0,0.0,1.0,4.430264,450.449272,6212.500000
1,-12.110077,-76.965640,48,2,1387.0,793.0,19.0,4.0,4.0,20.0,...,4.0,4.0,0.0,0.0,0.0,0.0,0.0,4.430264,450.254999,5969.718818
2,-12.079926,-76.923410,28,2,490.0,490.0,16.0,5.0,5.0,21.0,...,1.0,2.0,0.0,0.0,0.0,0.0,0.0,4.430017,450.476426,6946.312245
3,-12.207427,-76.937033,51,2,140.0,140.0,1.0,3.0,6.0,16.0,...,35.0,15.0,0.0,0.0,0.0,2.0,0.0,4.429770,450.562824,2200.000000
4,-12.144977,-76.989592,48,2,338.0,215.0,4.0,3.0,4.0,23.0,...,24.0,13.0,0.0,2.0,0.0,2.0,1.0,4.430583,450.444911,6286.390533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3641,-12.092924,-76.923858,18,2,371.0,355.0,12.0,4.0,4.0,15.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,4.430264,450.505801,4528.301887
3642,-12.085432,-76.957820,18,2,260.0,200.0,4.0,4.0,4.0,18.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,4.430264,450.533202,6057.692308
3643,-12.073441,-76.914661,18,2,700.0,450.0,15.0,4.0,4.0,19.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.430264,450.424587,6750.000000
3644,-12.841678,-76.526003,1,1,300.0,200.0,4.0,4.0,4.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.430477,450.686589,4980.000000


tree = BallTree(test_points, metric = 'haversine')

results = tree.query_radius(test_points, r=10, return_distance=True)


#From stackoverflow
from sklearn.neighbors import BallTree

ratio_tierra = 6371000
ratio_test = 1300000

test_points = [[-12.011108,-77.053447],[-12.003141, -77.06475]]
test_points_rad = np.array([[np.radians(x[0]), np.radians(x[1])] for x in test_points])

tree = BallTree(test_points_rad, metric='haversine')

ind, results = tree.query_radius(test_points_rad, r = ratio_test/ratio_tierra, return_distance=True)

print(ind)
print(results * ratio_tierra/1000)


def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Para calcular la distancia entre dos ubicaciones en la tierra.
    En kilometros.
    """
    r = 6371
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
    res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    return np.round(res, 4)

haversine_distance(-12.011108,-77.053447,-12.003141, -77.06475)


from sklearn.neighbors import BallTree
import numpy as np

rng = np.random.RandomState(1)
X = rng.random_sample((10,2))

tree = BallTree(X, leaf_size=1, metric='haversine')
dist, ind = tree.query(X[:1], k=2)
print(ind)
print(dist)